In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

In [ ]:
# prepare data
train_datagen_with_no_transforms = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

train_generator = train_datagen_with_no_transforms.flow_from_directory(
    '../input/emotion-detection-fer/train',
    target_size=(32,32),
    batch_size=32,
    class_mode="sparse"
)
validation_generator = val_datagen.flow_from_directory(
    '../input/emotion-detection-fer/test',
    target_size=(32,32),
    batch_size=32,
    class_mode="sparse"
)

In [ ]:
sample_data = next(validation_generator)
sample_data[0].shape

In [ ]:
nb_train_samples = train_generator.samples
nb_val_samples = validation_generator.samples
train_generator.samples

# ALEXNET SEQUENTIAL

In [ ]:
input_shape=(32,32, 3) 
num_classes = 7
# AlexNET model
seq_alex = models.Sequential()
#seq_alex.add() # input shape must be specified according to the input data dimension
#seq_alex.add()

seq_alex.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal'))
seq_alex.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None))

seq_alex.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
seq_alex.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None)) 

seq_alex.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

seq_alex.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

seq_alex.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

seq_alex.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None))

seq_alex.add(Flatten())
seq_alex.add(Dense(4096, activation= 'relu'))
seq_alex.add(Dense(4096, activation= 'relu'))
seq_alex.add(Dense(1000, activation= 'relu'))
seq_alex.add(Dense(num_classes, activation= 'softmax'))

seq_alex.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [ ]:
# compile model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)

seq_alex.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
# train model

# callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
callback_list = [ early_stopping_callback]

# train
history = seq_alex.fit_generator(train_generator, steps_per_epoch=nb_train_samples/2000, 
                    epochs=100, validation_data=validation_generator,
                    validation_steps=nb_val_samples/2000, callbacks=callback_list)

In [ ]:
layer_names = [layer.name for layer in seq_alex.layers]
layer_names

In [ ]:
seq_alex.layers

In [ ]:
layer_outputs = [layer.output for layer in seq_alex.layers]
layer_outputs

In [ ]:
feature_map_model = tf.keras.models.Model(seq_alex.input, layer_outputs)

In [ ]:
image_path= r"../input/emotion-detection-fer/test/happy/im1.png"
img = tf.keras.utils.load_img(image_path, target_size=(32, 32))  
input = tf.keras.utils.img_to_array(img)                           
input = input.reshape((1,) + input.shape)                   
input /= 255.0

In [ ]:
feature_maps = feature_map_model.predict(input)
feature_maps

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")

In [ ]:
feature_map.shape

In [ ]:
feature_map[0].shape

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
seq_alex.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)

#Defining the parameters
batch_size= 2000
epochs=100
learn_rate=.001

seq_alex.fit_generator(train_generator, epochs = epochs, validation_data = validation_generator, validation_steps = 250, callbacks = [lrr], verbose=1)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

# ALEXNET FUNCTIONAL API

In [ ]:
input_shape=(32,32,3)
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np 
# Do
# AlexNET model with functional API
inputs = tf.keras.layers.Input(shape=input_shape) # input shape must be specified according to the input data dimension
#x = tf.keras.layers.Conv2D()(inputs)
#x = tf.keras.layers.MaxPool2D()(x)

#functional_alex = models.Model(inputs=inputs, outputs=)

#X_input = Input(input_shape)   
X = Conv2D(96,(11,11),strides = 4,name="conv0")(inputs)
X = BatchNormalization(axis = 3 , name = "bn0")(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max0',padding='same')(X)
    
X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
X = BatchNormalization(axis = 3 ,name='bn1')(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max1',padding='same')(X)
    
X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
X = BatchNormalization(axis = 3, name = 'bn2')(X)
X = Activation('relu')(X)
    
X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
X = BatchNormalization(axis = 3, name = 'bn3')(X)
X = Activation('relu')(X)
    
X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
X = BatchNormalization(axis = 3, name = 'bn4')(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max2',padding='same')(X)
    
X = Flatten()(X)
    
X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
X = Dense(1000,activation='softmax',name = 'fc2')(X) 
    
model = Model(inputs = inputs, outputs = X, name='AlexNet')

In [ ]:
# compile model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
# train model

# callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
callback_list = [ early_stopping_callback]

# train
history = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples/2000, 
                    epochs=100, validation_data=validation_generator,
                    validation_steps=nb_val_samples/2000, callbacks=callback_list)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)

#Defining the parameters
batch_size= 2000
epochs=100
learn_rate=.001

model.fit_generator(train_generator, epochs = epochs, validation_data = validation_generator, validation_steps = 250, callbacks = [lrr], verbose=1)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

# ALEXNET SEQUENTIAL API

In [ ]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

In [ ]:
AlexNet.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [ ]:
#Defining the parameters
batch_size= 2000
epochs=100
learn_rate=.001

In [ ]:
#Training the model
AlexNet.fit_generator(train_generator, epochs = epochs, validation_data = validation_generator, validation_steps = 250, callbacks = [lrr], verbose=1)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
predictions = AlexNet.predict(validation_generator)
predictions = predictions.reshape(1,-1)[0]
predictions[:15]

In [ ]:
correct = np.nonzero(predictions == validation_generator)[0]
incorrect = np.nonzero(predictions != validation_generator)[0]

In [ ]:
i = 0
for c in correct[:6]:
    plt.subplot(3,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[c].reshape(150,150), cmap="gray", interpolation='none')
    plt.title("Predicted Class {},Actual Class {}".format(predictions[c], validation_generator[c]))
    plt.tight_layout()
    i += 1

In [ ]:
#Iterate through all the layers of the model
for layer in model.layers:
    if 'Conv' in layer.name:
        weights, bias= layer.get_weights()
        print(layer.name, filters.shape)
        
        #normalize filter values between  0 and 1 for visualization
        f_min, f_max = weights.min(), weights.max()
        filters = (weights - f_min) / (f_max - f_min)  
        print(filters.shape[3])
        filter_cnt=1
        
        #plotting all the filters
        for i in range(filters.shape[3]):
            #get the filters
            filt=filters[:,:,:, i]
            #plotting each of the channel, color image RGB channels
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:,:, j])
                filter_cnt+=1
        plt.show()


In [ ]:
img_path='../input/emotion-detection-fer/test/happy/im1.png' 
# Define a new Model, Input= image 
# Output= intermediate representations for all layers in the  
# previous model after the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
#Load the input image
img = tf.keras.utils.load_img(img_path, target_size=(32, 32))
# Convert ht image to Array of dimension (32,32,3)
x   = tf.keras.utils.img_to_array(img)                           
x   = x.reshape((1,) + x.shape)
# Rescale by 1/255
x /= 255.0
# Let's run input image through our vislauization network
# to obtain all intermediate representations for the image.
successive_feature_maps = visualization_model.predict(x)
# Retrieve are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    print(feature_map.shape)
    if len(feature_map.shape) == 4:
        # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers
   
        n_features = feature_map.shape[-1]  # number of features in the feature map
        size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    # Postprocess the feature to be visually palatable
    for i in range(n_features):
        x  = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std ()
        x *=  64
        x += 128
        x  = np.clip(x, 0, 255).astype('uint8')
        # Tile each filter into a horizontal grid
        display_grid[:, i * size : (i + 1) * size] = x
# Display the grid
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )